In [ ]:
# Install TensorFlow
# !pip install -q tensorflow-gpu==2.0.0-beta1

#try:
#  %tensorflow_version 2.x  # Colab only.
#except Exception:
#  pass

import tensorflow as tf
print(tf.__version__)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab
import string
import math


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#with open('/content/drive/My Drive/foo.txt', 'w') as f:
#  f.write('Hello Google Drive!')
#!cat /content/drive/My\ Drive/foo.txt

In [ ]:
#from google.colab import files
#uploaded = files.upload()

In [ ]:
#import io
#Data_Frame = pd.read_csv(io.BytesIO(uploaded['MITLL_Correlated_RNN_Data.csv']))

# Dataset is now stored in a Pandas Dataframe

In [ ]:
#!pwd

# New Section : RNN

In [ ]:
# More imports
from tensorflow import keras
from tensorflow.keras.layers import Input, SimpleRNN, GRU, LSTM, Dense, Flatten, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Data_Frame_1 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_NoisyData_20sec_NEW_1.csv')#, header=None)  
Data_Frame_1 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_20sec_NEW_1.csv')#, header=None)  


In [ ]:
Data_Frame_1.head()

#list(Data_Frame)[14]

#Data_Frame[list(Data_Frame)[14]]


In [ ]:
T = 20
T1 = 20

#---------------------
#T2 = 70   # this is to only consider examples of class 0 that are far from each other
#---------------------

Ratio_Times = 1     # This is the ratio fof the number of examples in two classes
D = 12


In [ ]:

## Data_Frame_1:

### DETERMINE the number of conflict versus non-conflict case.

#------------------------

Y_TEMP_1 = []

col_list_1 = list(Data_Frame_1)
N_enc_train_1 = (np.array(len(col_list_1)-2)/13).astype(int) # - 5


for i_enc in range(N_enc_train_1):    # Number of examples

    if Data_Frame_1[col_list_1[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:
       
       for i_t in range(150):

              y = Data_Frame_1[col_list_1[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

              if (y==0 or y==1) and i_t>T1-2:

                  Y_TEMP_1.append(y)

         
Y_TEMP_1 = np.array(Y_TEMP_1)


N_ratio_1 = np.sum(Y_TEMP_1)/(len(Y_TEMP_1)-np.sum(Y_TEMP_1))
N_ratio_1

In [ ]:
N_enc_train_1

In [ ]:
### Data_Frame_1:

### Prepare Data

#------------------------

X = []
Y = []

col_list_1 = list(Data_Frame_1)

for i_enc in range(N_enc_train_1):    # Number of examples

    if Data_Frame_1[col_list_1[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

      for i_t in range(150):
        
        #=============================== Prepare for Future classification case

        y = Data_Frame_1[col_list_1[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

        x = np.zeros(shape=(T,D))

        if (y==0 or y==1) and i_t>T1-2:

           if y==1:

              x[:,0] =  Data_Frame_1[col_list_1[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,1] =  Data_Frame_1[col_list_1[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,2] =  Data_Frame_1[col_list_1[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,3] =  Data_Frame_1[col_list_1[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,4] =  Data_Frame_1[col_list_1[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,5] =  Data_Frame_1[col_list_1[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,6] =  Data_Frame_1[col_list_1[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,7] =  Data_Frame_1[col_list_1[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,8] =  Data_Frame_1[col_list_1[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,9] =  Data_Frame_1[col_list_1[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,10] =  Data_Frame_1[col_list_1[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,11] =  Data_Frame_1[col_list_1[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


              X.append(x)
              Y.append(y)


           elif y==0 and (np.random.uniform(low=0.0,high=1.0)<N_ratio_1*Ratio_Times):  # and i_t<T2:    # this "and i_t<T2" is to have only examples far away *****


              x[:,0] =  Data_Frame_1[col_list_1[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,1] =  Data_Frame_1[col_list_1[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,2] =  Data_Frame_1[col_list_1[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,3] =  Data_Frame_1[col_list_1[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,4] =  Data_Frame_1[col_list_1[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,5] =  Data_Frame_1[col_list_1[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,6] =  Data_Frame_1[col_list_1[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,7] =  Data_Frame_1[col_list_1[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,8] =  Data_Frame_1[col_list_1[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,9] =  Data_Frame_1[col_list_1[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,10] =  Data_Frame_1[col_list_1[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,11] =  Data_Frame_1[col_list_1[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


              X.append(x)
              Y.append(y)


        #=================================================================
        #-------------------------Delete Variables
           del x
           del y
  
        #-------------------------

    


In [ ]:
len(X)

In [ ]:
len(Y)

In [ ]:
Y_a_1 = np.array(Y)
np.sum(Y_a_1)

In [ ]:
#Data_Frame_2 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_NoisyData_20sec_NEW_2.csv')#, header=None)  
Data_Frame_2 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_20sec_NEW_2.csv')#, header=None)  


In [ ]:
Data_Frame_2.head()

#list(Data_Frame_2)[14]

#Data_Frame_2[list(Data_Frame_2)[14]]


In [ ]:
col_list_2 = list(Data_Frame_2)

Data_Frame_2[col_list_2[14]][:]

In [ ]:
### Data_Frame_2:

### DETERMINE the number of conflict versus non-conflict case.

#------------------------

Y_TEMP_2 = []

col_list_2 = list(Data_Frame_2)
N_enc_train_2 = (np.array(len(col_list_2)-2)/13).astype(int) #- 5


for i_enc in range(N_enc_train_2):    # Number of examples

    if Data_Frame_2[col_list_2[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

      for i_t in range(150):

        y = Data_Frame_2[col_list_2[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

        if (y==0 or y==1) and i_t>T1-2:

            Y_TEMP_2.append(y)

Y_TEMP_2 = np.array(Y_TEMP_2)


N_ratio_2 = np.sum(Y_TEMP_2)/(len(Y_TEMP_2)-np.sum(Y_TEMP_2))
N_ratio_2

In [ ]:
N_enc_train_2

In [ ]:
### Data_Frame_2:

### Prepare Data

#------------------------

col_list_2 = list(Data_Frame_2)

for i_enc in range(N_enc_train_2):    # Number of examples

    if Data_Frame_2[col_list_2[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

      for i_t in range(150):
        
        #=============================== Prepare for Future classification case

        y = Data_Frame_2[col_list_2[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

        x = np.zeros(shape=(T,D))

        if (y==0 or y==1) and i_t>T1-2:

           if y==1:

              x[:,0] =  Data_Frame_2[col_list_2[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,1] =  Data_Frame_2[col_list_2[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,2] =  Data_Frame_2[col_list_2[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,3] =  Data_Frame_2[col_list_2[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,4] =  Data_Frame_2[col_list_2[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,5] =  Data_Frame_2[col_list_2[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,6] =  Data_Frame_2[col_list_2[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,7] =  Data_Frame_2[col_list_2[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,8] =  Data_Frame_2[col_list_2[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,9] =  Data_Frame_2[col_list_2[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,10] =  Data_Frame_2[col_list_2[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,11] =  Data_Frame_2[col_list_2[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


              X.append(x)
              Y.append(y)


           elif y==0 and (np.random.uniform(low=0.0,high=1.0)<N_ratio_2*Ratio_Times):  # and i_t<T2:    # this "and i_t<T2" is to have only examples far away **********


              x[:,0] =  Data_Frame_2[col_list_2[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,1] =  Data_Frame_2[col_list_2[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,2] =  Data_Frame_2[col_list_2[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,3] =  Data_Frame_2[col_list_2[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,4] =  Data_Frame_2[col_list_2[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,5] =  Data_Frame_2[col_list_2[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,6] =  Data_Frame_2[col_list_2[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,7] =  Data_Frame_2[col_list_2[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,8] =  Data_Frame_2[col_list_2[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,9] =  Data_Frame_2[col_list_2[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,10] =  Data_Frame_2[col_list_2[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,11] =  Data_Frame_2[col_list_2[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


              X.append(x)
              Y.append(y)


        #=================================================================
        #-------------------------Delete Variables
           del x
           del y
  
        #-------------------------


    

In [ ]:
len(X)

In [ ]:
len(Y)

In [ ]:
Y_a_2 = np.array(Y)
np.sum(Y_a_2)

In [ ]:

#Data_Frame_3 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_NoisyData_20sec_NEW_3.csv')#, header=None)  
Data_Frame_3 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_20sec_NEW_3.csv')#, header=None)  


In [ ]:
Data_Frame_3.head()

#list(Data_Frame_3)[14]

#Data_Frame_3[list(Data_Frame_3)[14]]


In [ ]:
### Data_Frame_3:

### DETERMINE the number of conflict versus non-conflict case.

#------------------------

Y_TEMP_3 = []

col_list_3 = list(Data_Frame_3)
N_enc_train_3 = (np.array(len(col_list_3)-2)/13).astype(int) #- 5


for i_enc in range(N_enc_train_3):    # Number of examples

    if Data_Frame_3[col_list_3[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

      for i_t in range(150):

        y = Data_Frame_3[col_list_3[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

        if (y==0 or y==1) and i_t>T1-2:

            Y_TEMP_3.append(y)
    
Y_TEMP_3 = np.array(Y_TEMP_3)
len(Y_TEMP_3)


N_ratio_3 = np.sum(Y_TEMP_3)/(len(Y_TEMP_3)-np.sum(Y_TEMP_3))
N_ratio_3

In [ ]:
N_enc_train_3

In [ ]:
### Data_Frame_3;

### Prepare Data

#------------------------

col_list_3 = list(Data_Frame_3)

for i_enc in range(N_enc_train_3):    # Number of examples
 
    if Data_Frame_3[col_list_3[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:
       
       for i_t in range(150):
        
          #=============================== Prepare for Future classification case

          y = Data_Frame_3[col_list_3[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

          x = np.zeros(shape=(T,D))

          if (y==0 or y==1) and i_t>T1-2:

             if y==1:

                x[:,0] =  Data_Frame_3[col_list_3[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,1] =  Data_Frame_3[col_list_3[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,2] =  Data_Frame_3[col_list_3[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,3] =  Data_Frame_3[col_list_3[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,4] =  Data_Frame_3[col_list_3[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,5] =  Data_Frame_3[col_list_3[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,6] =  Data_Frame_3[col_list_3[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,7] =  Data_Frame_3[col_list_3[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,8] =  Data_Frame_3[col_list_3[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,9] =  Data_Frame_3[col_list_3[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,10] =  Data_Frame_3[col_list_3[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,11] =  Data_Frame_3[col_list_3[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


                X.append(x)
                Y.append(y)


             elif y==0 and (np.random.uniform(low=0.0,high=1.0)<N_ratio_3*Ratio_Times): # and i_t<T2:    # this "and i_t<T2" is to have only examples far away **********


                x[:,0] =  Data_Frame_3[col_list_3[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,1] =  Data_Frame_3[col_list_3[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,2] =  Data_Frame_3[col_list_3[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,3] =  Data_Frame_3[col_list_3[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,4] =  Data_Frame_3[col_list_3[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,5] =  Data_Frame_3[col_list_3[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,6] =  Data_Frame_3[col_list_3[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,7] =  Data_Frame_3[col_list_3[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,8] =  Data_Frame_3[col_list_3[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,9] =  Data_Frame_3[col_list_3[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,10] =  Data_Frame_3[col_list_3[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,11] =  Data_Frame_3[col_list_3[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


                X.append(x)
                Y.append(y)


          #=================================================================
          #-------------------------Delete Variables
             del x
             del y
  
          #-------------------------

  

In [ ]:
len(X)

In [ ]:
len(Y)

In [ ]:
Y_a_3 = np.sum(np.array(Y))
Y_a_3

In [ ]:
#Data_Frame_4 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_NoisyData_20sec_NEW_4.csv')#, header=None)  
Data_Frame_4 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_20sec_NEW_4.csv')#, header=None)  



In [ ]:
Data_Frame_4.head()

#list(Data_Frame_4)[14]

#Data_Frame_4[list(Data_Frame_4)[14]]


In [ ]:
### Data_Frame_4:

### DETERMINE the number of conflict versus non-conflict case.

#------------------------

Y_TEMP_4 = []

col_list_4 = list(Data_Frame_4)
N_enc_train_4 = (np.array(len(col_list_4)-2)/13).astype(int) #- 5


for i_enc in range(N_enc_train_4):    # Number of examples

    if Data_Frame_4[col_list_4[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

       for i_t in range(150):

           y = Data_Frame_4[col_list_4[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

           if (y==0 or y==1) and i_t>T1-2:

               Y_TEMP_4.append(y)


Y_TEMP_4 = np.array(Y_TEMP_4)
len(Y_TEMP_4)


N_ratio_4 = np.sum(Y_TEMP_4)/(len(Y_TEMP_4)-np.sum(Y_TEMP_4))
N_ratio_4

In [ ]:
N_enc_train_4

In [ ]:
### Data_Frame_4:

### Prepare Data

#------------------------

col_list_4 = list(Data_Frame_4)

for i_enc in range(N_enc_train_4):    # Number of examples

    if Data_Frame_4[col_list_4[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

       for i_t in range(150):
        
          #=============================== Prepare for Future classification case

          y = Data_Frame_4[col_list_4[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

          x = np.zeros(shape=(T,D))

          if (y==0 or y==1) and i_t>T1-2:

             if y==1:

                x[:,0] =  Data_Frame_4[col_list_4[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,1] =  Data_Frame_4[col_list_4[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,2] =  Data_Frame_4[col_list_4[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,3] =  Data_Frame_4[col_list_4[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,4] =  Data_Frame_4[col_list_4[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,5] =  Data_Frame_4[col_list_4[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,6] =  Data_Frame_4[col_list_4[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,7] =  Data_Frame_4[col_list_4[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,8] =  Data_Frame_4[col_list_4[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,9] =  Data_Frame_4[col_list_4[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,10] =  Data_Frame_4[col_list_4[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,11] =  Data_Frame_4[col_list_4[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


                X.append(x)
                Y.append(y)


             elif y==0 and (np.random.uniform(low=0.0,high=1.0)<N_ratio_4*Ratio_Times):  # and i_t<T2:    # this "and i_t<T2" is to have only examples far away **********


                x[:,0] =  Data_Frame_4[col_list_4[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,1] =  Data_Frame_4[col_list_4[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,2] =  Data_Frame_4[col_list_4[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,3] =  Data_Frame_4[col_list_4[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,4] =  Data_Frame_4[col_list_4[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,5] =  Data_Frame_4[col_list_4[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,6] =  Data_Frame_4[col_list_4[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,7] =  Data_Frame_4[col_list_4[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,8] =  Data_Frame_4[col_list_4[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,9] =  Data_Frame_4[col_list_4[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,10] =  Data_Frame_4[col_list_4[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,11] =  Data_Frame_4[col_list_4[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


                X.append(x)
                Y.append(y)


          #=================================================================
          #-------------------------Delete Variables
             del x
             del y
  
          #-------------------------

    


In [ ]:
len(X)

In [ ]:
len(Y)

In [ ]:
Y_a_4 = np.sum(np.array(Y))
Y_a_4

In [ ]:
#Data_Frame_5 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_NoisyData_20sec_NEW_5.csv')#, header=None)  
Data_Frame_5 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_20sec_NEW_5.csv')#, header=None)  



In [ ]:
Data_Frame_5.head()

#list(Data_Frame_5)[14]

#Data_Frame_5[list(Data_Frame_5)[14]]


In [ ]:
### Data_Frame_5:

### DETERMINE the number of conflict versus non-conflict case.

#------------------------

Y_TEMP_5 = []

col_list_5 = list(Data_Frame_5)
N_enc_train_5 = (np.array(len(col_list_5)-2)/13).astype(int) #- 5


for i_enc in range(N_enc_train_5):    # Number of examples

    if Data_Frame_5[col_list_5[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:
       
       for i_t in range(150):

           y = Data_Frame_5[col_list_5[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

           if (y==0 or y==1) and i_t>T1-2:

               Y_TEMP_5.append(y)

    
Y_TEMP_5 = np.array(Y_TEMP_5)
len(Y_TEMP_5)


N_ratio_5 = np.sum(Y_TEMP_5)/(len(Y_TEMP_5)-np.sum(Y_TEMP_5))
N_ratio_5

In [ ]:
N_enc_train_5

In [ ]:
### Data_Frame_5:

### Prepare Data

#------------------------

col_list_5 = list(Data_Frame_5)

for i_enc in range(N_enc_train_5):    # Number of examples

    if Data_Frame_5[col_list_5[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

       for i_t in range(150):
        
          #=============================== Prepare for Future classification case

          y = Data_Frame_5[col_list_5[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

          x = np.zeros(shape=(T,D))

          if (y==0 or y==1) and i_t>T1-2:

             if y==1:

                x[:,0] =  Data_Frame_5[col_list_5[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,1] =  Data_Frame_5[col_list_5[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,2] =  Data_Frame_5[col_list_5[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,3] =  Data_Frame_5[col_list_5[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,4] =  Data_Frame_5[col_list_5[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,5] =  Data_Frame_5[col_list_5[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,6] =  Data_Frame_5[col_list_5[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,7] =  Data_Frame_5[col_list_5[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,8] =  Data_Frame_5[col_list_5[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,9] =  Data_Frame_5[col_list_5[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,10] =  Data_Frame_5[col_list_5[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,11] =  Data_Frame_5[col_list_5[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


                X.append(x)
                Y.append(y)


             elif y==0 and (np.random.uniform(low=0.0,high=1.0)<N_ratio_5*Ratio_Times):  # and i_t<T2:    # this "and i_t<T2" is to have only examples far away **********


                x[:,0] =  Data_Frame_5[col_list_5[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,1] =  Data_Frame_5[col_list_5[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,2] =  Data_Frame_5[col_list_5[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,3] =  Data_Frame_5[col_list_5[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,4] =  Data_Frame_5[col_list_5[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,5] =  Data_Frame_5[col_list_5[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,6] =  Data_Frame_5[col_list_5[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,7] =  Data_Frame_5[col_list_5[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,8] =  Data_Frame_5[col_list_5[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,9] =  Data_Frame_5[col_list_5[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,10] =  Data_Frame_5[col_list_5[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,11] =  Data_Frame_5[col_list_5[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


                X.append(x)
                Y.append(y)


          #=================================================================
          #-------------------------Delete Variables
             del x
             del y
  
          #-------------------------

    



In [ ]:
len(X)

In [ ]:
len(Y)

In [ ]:
Y_a_5 = np.sum(np.array(Y))
Y_a_5

In [ ]:
#Data_Frame_6 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_NoisyData_20sec_NEW_6.csv')#, header=None)  
Data_Frame_6 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_20sec_NEW_6.csv')#, header=None)  



In [ ]:
Data_Frame_6.head()

#list(Data_Frame_6)[14]

#Data_Frame_6[list(Data_Frame_6)[14]]


In [ ]:
### Data_Frame_6:

### DETERMINE the number of conflict versus non-conflict case.

#------------------------

Y_TEMP_6 = []

col_list_6 = list(Data_Frame_6)
N_enc_train_6 = (np.array(len(col_list_6)-2)/13).astype(int) #- 5


for i_enc in range(N_enc_train_6):    # Number of examples

    if Data_Frame_6[col_list_6[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

       for i_t in range(150):

          y = Data_Frame_6[col_list_6[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

          if (y==0 or y==1) and i_t>T1-2:

              Y_TEMP_6.append(y)

    
Y_TEMP_6 = np.array(Y_TEMP_6)
len(Y_TEMP_6)


N_ratio_6 = np.sum(Y_TEMP_6)/(len(Y_TEMP_6)-np.sum(Y_TEMP_6))
N_ratio_6

In [ ]:
N_enc_train_6

In [ ]:
### Data_Frame_6:

### Prepare Data

#------------------------

col_list_6 = list(Data_Frame_6)

for i_enc in range(N_enc_train_6):    # Number of examples

    if Data_Frame_6[col_list_6[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

       for i_t in range(150):
        
          #=============================== Prepare for Future classification case

          y = Data_Frame_6[col_list_6[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

          x = np.zeros(shape=(T,D))

          if (y==0 or y==1) and i_t>T1-2:

             if y==1:

                x[:,0] =  Data_Frame_6[col_list_6[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,1] =  Data_Frame_6[col_list_6[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,2] =  Data_Frame_6[col_list_6[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,3] =  Data_Frame_6[col_list_6[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,4] =  Data_Frame_6[col_list_6[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,5] =  Data_Frame_6[col_list_6[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,6] =  Data_Frame_6[col_list_6[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,7] =  Data_Frame_6[col_list_6[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,8] =  Data_Frame_6[col_list_6[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,9] =  Data_Frame_6[col_list_6[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,10] =  Data_Frame_6[col_list_6[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,11] =  Data_Frame_6[col_list_6[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


                X.append(x)
                Y.append(y)


             elif y==0 and (np.random.uniform(low=0.0,high=1.0)<N_ratio_6*Ratio_Times):  # and i_t<T2:    # this "and i_t<T2" is to have only examples far away **********


                x[:,0] =  Data_Frame_6[col_list_6[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,1] =  Data_Frame_6[col_list_6[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,2] =  Data_Frame_6[col_list_6[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,3] =  Data_Frame_6[col_list_6[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,4] =  Data_Frame_6[col_list_6[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,5] =  Data_Frame_6[col_list_6[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,6] =  Data_Frame_6[col_list_6[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,7] =  Data_Frame_6[col_list_6[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,8] =  Data_Frame_6[col_list_6[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,9] =  Data_Frame_6[col_list_6[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,10] =  Data_Frame_6[col_list_6[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,11] =  Data_Frame_6[col_list_6[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


                X.append(x)
                Y.append(y)


          #=================================================================
          #-------------------------Delete Variables
             del x
             del y
  
          #-------------------------

    


In [ ]:
len(X)

In [ ]:
len(Y)

In [ ]:
Y_a_6 = np.sum(np.array(Y))
Y_a_6

In [ ]:
#Data_Frame_7 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_NoisyData_20sec_NEW_7.csv')#, header=None)  
Data_Frame_7 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_20sec_NEW_7.csv')#, header=None)  



In [ ]:
Data_Frame_7.head()

#list(Data_Frame_7)[14]

#Data_Frame_7[list(Data_Frame_7)[14]]


In [ ]:
### Data_Frame_7:

### DETERMINE the number of conflict versus non-conflict case.

#------------------------

Y_TEMP_7 = []

col_list_7 = list(Data_Frame_7)
N_enc_train_7 = (np.array(len(col_list_7)-2)/13).astype(int) #- 5


for i_enc in range(N_enc_train_7):    # Number of examples

    if Data_Frame_7[col_list_7[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

       for i_t in range(150):

          y = Data_Frame_7[col_list_7[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

          if (y==0 or y==1) and i_t>T1-2:

              Y_TEMP_7.append(y)


Y_TEMP_7 = np.array(Y_TEMP_7)
len(Y_TEMP_7)


N_ratio_7 = np.sum(Y_TEMP_7)/(len(Y_TEMP_7)-np.sum(Y_TEMP_7))
N_ratio_7

In [ ]:
N_enc_train_7

In [ ]:
### Data_Frame_7:

### Prepare Data

#------------------------

col_list_7 = list(Data_Frame_7)

for i_enc in range(N_enc_train_7):    # Number of examples

    if Data_Frame_7[col_list_7[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:
       
       for i_t in range(150):
        
          #=============================== Prepare for Future classification case

          y = Data_Frame_7[col_list_7[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

          x = np.zeros(shape=(T,D))

          if (y==0 or y==1) and i_t>T1-2:

             if y==1:

                x[:,0] =  Data_Frame_7[col_list_7[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,1] =  Data_Frame_7[col_list_7[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,2] =  Data_Frame_7[col_list_7[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,3] =  Data_Frame_7[col_list_7[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,4] =  Data_Frame_7[col_list_7[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,5] =  Data_Frame_7[col_list_7[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,6] =  Data_Frame_7[col_list_7[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,7] =  Data_Frame_7[col_list_7[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,8] =  Data_Frame_7[col_list_7[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,9] =  Data_Frame_7[col_list_7[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,10] =  Data_Frame_7[col_list_7[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,11] =  Data_Frame_7[col_list_7[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


                X.append(x)
                Y.append(y)


             elif y==0 and (np.random.uniform(low=0.0,high=1.0)<N_ratio_7*Ratio_Times):  # and i_t<T2:    # this "and i_t<T2" is to have only examples far away **********


                x[:,0] =  Data_Frame_7[col_list_7[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,1] =  Data_Frame_7[col_list_7[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,2] =  Data_Frame_7[col_list_7[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,3] =  Data_Frame_7[col_list_7[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,4] =  Data_Frame_7[col_list_7[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,5] =  Data_Frame_7[col_list_7[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,6] =  Data_Frame_7[col_list_7[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,7] =  Data_Frame_7[col_list_7[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,8] =  Data_Frame_7[col_list_7[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,9] =  Data_Frame_7[col_list_7[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,10] =  Data_Frame_7[col_list_7[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
                x[:,11] =  Data_Frame_7[col_list_7[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


                X.append(x)
                Y.append(y)


          #=================================================================
          #-------------------------Delete Variables
             del x
             del y
  
          #-------------------------

    


In [ ]:
len(X)

In [ ]:
len(Y)

In [ ]:
Y_a_7 = np.sum(np.array(Y))
Y_a_7

In [ ]:
#Data_Frame_8 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_NoisyData_20sec_NEW_8.csv')#, header=None)  
Data_Frame_8 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_20sec_NEW_8.csv')#, header=None)  



In [ ]:
Data_Frame_8.head()

#list(Data_Frame_8)[14]

#Data_Frame_8[list(Data_Frame_8)[14]]


In [ ]:
### Data_Frame_8:

### DETERMINE the number of conflict versus non-conflict case.

#------------------------

Y_TEMP_8 = []

col_list_8 = list(Data_Frame_8)
N_enc_train_8 = (np.array(len(col_list_8)-2)/13).astype(int) #- 5


for i_enc in range(N_enc_train_8):    # Number of examples

    if Data_Frame_8[col_list_8[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

       for i_t in range(150):

          y = Data_Frame_8[col_list_8[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

          if (y==0 or y==1) and i_t>T1-2:

              Y_TEMP_8.append(y)

    
Y_TEMP_8 = np.array(Y_TEMP_8)
len(Y_TEMP_8)


N_ratio_8 = np.sum(Y_TEMP_8)/(len(Y_TEMP_8)-np.sum(Y_TEMP_8))
N_ratio_8

In [ ]:
N_enc_train_8

In [ ]:
### Data_Frame_8:

### Prepare Data

#------------------------

col_list_8 = list(Data_Frame_8)

for i_enc in range(N_enc_train_8):    # Number of examples

    if Data_Frame_8[col_list_8[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

      for i_t in range(150):
        
        #=============================== Prepare for Future classification case

        y = Data_Frame_8[col_list_8[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

        x = np.zeros(shape=(T,D))

        if (y==0 or y==1) and i_t>T1-2:

           if y==1:

              x[:,0] =  Data_Frame_8[col_list_8[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,1] =  Data_Frame_8[col_list_8[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,2] =  Data_Frame_8[col_list_8[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,3] =  Data_Frame_8[col_list_8[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,4] =  Data_Frame_8[col_list_8[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,5] =  Data_Frame_8[col_list_8[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,6] =  Data_Frame_8[col_list_8[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,7] =  Data_Frame_8[col_list_8[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,8] =  Data_Frame_8[col_list_8[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,9] =  Data_Frame_8[col_list_8[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,10] =  Data_Frame_8[col_list_8[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,11] =  Data_Frame_8[col_list_8[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


              X.append(x)
              Y.append(y)


           elif y==0 and (np.random.uniform(low=0.0,high=1.0)<N_ratio_8*Ratio_Times):  # and i_t<T2:    # this "and i_t<T2" is to have only examples far away **********


              x[:,0] =  Data_Frame_8[col_list_8[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,1] =  Data_Frame_8[col_list_8[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,2] =  Data_Frame_8[col_list_8[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,3] =  Data_Frame_8[col_list_8[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,4] =  Data_Frame_8[col_list_8[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,5] =  Data_Frame_8[col_list_8[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,6] =  Data_Frame_8[col_list_8[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,7] =  Data_Frame_8[col_list_8[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,8] =  Data_Frame_8[col_list_8[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,9] =  Data_Frame_8[col_list_8[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,10] =  Data_Frame_8[col_list_8[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,11] =  Data_Frame_8[col_list_8[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


              X.append(x)
              Y.append(y)


        #=================================================================
        #-------------------------Delete Variables
           del x
           del y
  
        #-------------------------

    


In [ ]:
len(X)

In [ ]:
len(Y)

In [ ]:
Y_a_8 = np.sum(np.array(Y))
Y_a_8

In [ ]:
#Data_Frame_9 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_NoisyData_20sec_NEW_9.csv')#, header=None)  
Data_Frame_9 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_20sec_NEW_9.csv')#, header=None)  



In [ ]:
Data_Frame_9.head()

#list(Data_Frame_9)[14]

#Data_Frame_9[list(Data_Frame_9)[14]]


In [ ]:
### Data_Frame_9:

### DETERMINE the number of conflict versus non-conflict case.

#------------------------

Y_TEMP_9 = []


col_list_9 = list(Data_Frame_9)
N_enc_train_9 = (np.array(len(col_list_9)-2)/13).astype(int) #- 5


for i_enc in range(N_enc_train_9):    # Number of examples

    if Data_Frame_9[col_list_9[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

      for i_t in range(150):

        y = Data_Frame_9[col_list_9[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

        if (y==0 or y==1) and i_t>T1-2:

            Y_TEMP_9.append(y)


Y_TEMP_9 = np.array(Y_TEMP_9)
len(Y_TEMP_9)


N_ratio_9 = np.sum(Y_TEMP_9)/(len(Y_TEMP_9)-np.sum(Y_TEMP_9))
N_ratio_9

In [ ]:
N_enc_train_9

In [ ]:
### Data_Frame_9:

### Prepare Data

#------------------------

col_list_9 = list(Data_Frame_9)

for i_enc in range(N_enc_train_9):    # Number of examples

    if Data_Frame_9[col_list_9[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

      for i_t in range(150):
        
        #=============================== Prepare for Future classification case

        y = Data_Frame_9[col_list_9[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

        x = np.zeros(shape=(T,D))

        if (y==0 or y==1) and i_t>T1-2:

           if y==1:

              x[:,0] =  Data_Frame_9[col_list_9[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,1] =  Data_Frame_9[col_list_9[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,2] =  Data_Frame_9[col_list_9[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,3] =  Data_Frame_9[col_list_9[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,4] =  Data_Frame_9[col_list_9[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,5] =  Data_Frame_9[col_list_9[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,6] =  Data_Frame_9[col_list_9[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,7] =  Data_Frame_9[col_list_9[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,8] =  Data_Frame_9[col_list_9[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,9] =  Data_Frame_9[col_list_9[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,10] =  Data_Frame_9[col_list_9[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,11] =  Data_Frame_9[col_list_9[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


              X.append(x)
              Y.append(y)


           elif y==0 and (np.random.uniform(low=0.0,high=1.0)<N_ratio_9*Ratio_Times):  # and i_t<T2:    # this "and i_t<T2" is to have only examples far away **********


              x[:,0] =  Data_Frame_9[col_list_9[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,1] =  Data_Frame_9[col_list_9[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,2] =  Data_Frame_9[col_list_9[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,3] =  Data_Frame_9[col_list_9[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,4] =  Data_Frame_9[col_list_9[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,5] =  Data_Frame_9[col_list_9[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,6] =  Data_Frame_9[col_list_9[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,7] =  Data_Frame_9[col_list_9[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,8] =  Data_Frame_9[col_list_9[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,9] =  Data_Frame_9[col_list_9[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,10] =  Data_Frame_9[col_list_9[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,11] =  Data_Frame_9[col_list_9[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


              X.append(x)
              Y.append(y)


        #=================================================================
        #-------------------------Delete Variables
           del x
           del y
  
        #-------------------------

    


In [ ]:
len(X)

In [ ]:
len(Y)

In [ ]:
Y_a_9 = np.sum(np.array(Y))
Y_a_9

In [ ]:
### Convert list to numpy array to feed into RNN

X_train = np.array(X)
Y_train = np.array(Y)


In [ ]:
np.sum(Y_train)      ## Number of examples for class 1 (NMAC class)

In [ ]:
len(Y_train)-np.sum(Y_train)    ## Number of examples for class 0

In [ ]:
#Data_Frame_X = pd.DataFrame(X_train)
#Data_Frame_Y = pd.DataFrame(Y_train)

#Data_Frame_X.to_csv('/content/drive/My Drive/X_train.csv')
#Data_Frame_Y.to_csv('/content/drive/My Drive/Y_train.csv')


In [ ]:
### Delete those variables that are not needed any more to use less RAM

del Data_Frame_1
del Data_Frame_2
del Data_Frame_3
del Data_Frame_4
del Data_Frame_5
del Data_Frame_6
del Data_Frame_7
del Data_Frame_8
del Data_Frame_9

del Y_TEMP_1
del Y_TEMP_2
del Y_TEMP_3
del Y_TEMP_4
del Y_TEMP_5
del Y_TEMP_6
del Y_TEMP_7
del Y_TEMP_8
del Y_TEMP_9

del col_list_1
del col_list_2
del col_list_3
del col_list_4
del col_list_5
del col_list_6
del col_list_7
del col_list_8
del col_list_9

del N_enc_train_1
del N_enc_train_2
del N_enc_train_3
del N_enc_train_4
del N_enc_train_5
del N_enc_train_6
del N_enc_train_7
del N_enc_train_8
del N_enc_train_9


del N_ratio_1
del N_ratio_2
del N_ratio_3
del N_ratio_4
del N_ratio_5
del N_ratio_6
del N_ratio_7
del N_ratio_8
del N_ratio_9


del Y_a_1
del Y_a_2
del Y_a_3
del Y_a_4
del Y_a_5
del Y_a_6
del Y_a_7
del Y_a_8
del Y_a_9




In [ ]:
np.sum(Y_train)

In [ ]:
len(Y_train)-np.sum(Y_train)

In [ ]:
# Now Train a Deep LSTM with Global Max Pooling
#inputs = np.expand_dims(X, -1)
inputs = X_train

# make the RNN
i = Input(shape=(T, D))

# method 1
#x = LSTM(20)(i)


#----------------------------------- Tanh activation is default

#x = LSTM(20, return_sequences= True)(i) 
#x = LSTM(20, return_sequences= True)(x) 
#x = LSTM(20, return_sequences= True)(x) 
#x = LSTM(20, return_sequences= True)(x) 
#x = LSTM(5, return_sequences= True)(x) 
#x = LSTM(5, return_sequences= True)(x) 


#x = GlobalMaxPool1D()(x)


#x = Dense(1, activation='sigmoid')(x)
#model = Model(i, x)

#----------------------------------

#----------------------------------- Sequential approach

layer_1 = keras.layers.LSTM(25, activation='sigmoid')   #, return_sequences= True) 
#layer_2 = keras.layers.LSTM(20, return_sequences= True)
#layer_3 = keras.layers.LSTM(20, return_sequences= True)
#layer_4 = keras.layers.LSTM(20, return_sequences= True)

#layer_5 = keras.layers.GlobalMaxPool1D()
layer_6 = keras.layers.Dense(1, activation='sigmoid')

#model = keras.Sequential([i, layer_1, layer_2, layer_3, layer_4, layer_5, layer_6])

model = keras.Sequential([i, layer_1, layer_6])

#----------------------------------

#----------------------------------- Sequential approach

#layer_1 = keras.layers.LSTM(20, return_sequences= True, activation='sigmoid') 
#layer_2 = keras.layers.LSTM(20, return_sequences= True, activation='sigmoid')
#layer_3 = keras.layers.LSTM(20, return_sequences= True, activation='sigmoid')
#layer_4 = keras.layers.LSTM(20, return_sequences= True, activation='sigmoid')

#layer_5 = keras.layers.GlobalMaxPool1D()
#layer_6 = keras.layers.Dense(1, activation='sigmoid')

#model = keras.Sequential([i, layer_1, layer_2, layer_3, layer_4, layer_5, layer_6])

#----------------------------------


#----------------------------------- Sigmoid activation

#x = LSTM(15, return_sequences= True, activation='sigmoid')(i)
#x = LSTM(15, return_sequences= True, activation='sigmoid')(x)
#x = LSTM(15, return_sequences= True, activation='sigmoid')(x)
#x = LSTM(15, return_sequences= True, activation='sigmoid')(x)


#x = GlobalMaxPool1D()(x)


#x = Dense(1, activation='sigmoid')(x)
#model = Model(i, x)

#----------------------------------

#----------------------------------- Sigmoid activation

#x = LSTM(15, return_sequences= True, activation='relu')(i)
#x = LSTM(15, return_sequences= True, activation='relu')(x)
#x = LSTM(15, return_sequences= True, activation='relu')(x)
#x = LSTM(15, return_sequences= True, activation='relu')(x)


#x = GlobalMaxPool1D()(x)


#x = Dense(1, activation='sigmoid')(x)
#model = Model(i, x)

#----------------------------------



#------------------------------------ Try this block to have the accuracy with a threshold 0.5

#tf.keras.metrics.BinaryAccuracy(
#    name="binary_accuracy", dtype=None, threshold=0.5
#)

model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0003),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

#model.compile(
#  loss='binary_crossentropy',
#  # optimizer='rmsprop',
##   optimizer='adam',
#   optimizer=Adam(lr=0.01),
#   #optimizer=SGD(lr=0.1, momentum=0.9),
#  metrics=['accuracy'],
#)


# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=50,
  validation_split=0.1,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0002),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

#model.compile(
#  loss='binary_crossentropy',
#  # optimizer='rmsprop',
##   optimizer='adam',
#   optimizer=Adam(lr=0.01),
#   #optimizer=SGD(lr=0.1, momentum=0.9),
#  metrics=['accuracy'],
#)


# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=10,
  validation_split=0.2,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0001),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

#model.compile(
#  loss='binary_crossentropy',
#  # optimizer='rmsprop',
##   optimizer='adam',
#   optimizer=Adam(lr=0.01),
#   #optimizer=SGD(lr=0.1, momentum=0.9),
#  metrics=['accuracy'],
#)


# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=10,
  validation_split=0.2,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0001),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

#model.compile(
#  loss='binary_crossentropy',
#  # optimizer='rmsprop',
##   optimizer='adam',
#   optimizer=Adam(lr=0.01),
#   #optimizer=SGD(lr=0.1, momentum=0.9),
#  metrics=['accuracy'],
#)


# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.2,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0001),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

#model.compile(
#  loss='binary_crossentropy',
#  # optimizer='rmsprop',
##   optimizer='adam',
#   optimizer=Adam(lr=0.01),
#   #optimizer=SGD(lr=0.1, momentum=0.9),
#  metrics=['accuracy'],
#)


# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.2,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0001),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

#model.compile(
#  loss='binary_crossentropy',
#  # optimizer='rmsprop',
##   optimizer='adam',
#   optimizer=Adam(lr=0.01),
#   #optimizer=SGD(lr=0.1, momentum=0.9),
#  metrics=['accuracy'],
#)


# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.2,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0001),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

#model.compile(
#  loss='binary_crossentropy',
#  # optimizer='rmsprop',
##   optimizer='adam',
#   optimizer=Adam(lr=0.01),
#   #optimizer=SGD(lr=0.1, momentum=0.9),
#  metrics=['accuracy'],
#)


# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.1,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0001),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.2,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0001),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.2,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0001),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.1,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0001),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.1,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0001),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.2,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.00008),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.2,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.00006),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.2,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.00004),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.2,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.00003),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.2,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.00001),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.3,
  #validation_split=0.5,
  )


In [ ]:
model.compile(
  loss='binary_crossentropy',
    #optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.000005),
   #optimizer=SGD(lr=0.000001, momentum=0.6),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=3,
  validation_split=0.2,
  #validation_split=0.5,
  )


In [ ]:
# Plot the loss
plt.plot(r1.history['loss'], label='loss')
plt.plot(r1.history['val_loss'], label='val_loss')
plt.legend()

plt.title('Loss', fontsize=15)

plt.savefig('Loss_20sec_Noisy_Far_0_Class.png', dpi=220, bbox_inches='tight')

In [ ]:
## Plot the accuracy too

#------------------------------------------Use this block if Keras accuracy used above
# Plot the accuracy too
plt.plot(r1.history['binary_accuracy'], label='acc')
plt.plot(r1.history['val_binary_accuracy'], label='val_acc')
plt.legend()

plt.title('Accuracy', fontsize=15)

plt.savefig('Accuracy_20sec_Noisy_Far_0_Class.png', dpi=220, bbox_inches='tight')

#------------------------------------------

#----------------------------------------------------
#plt.plot(r.history['accuracy'], label='acc')
#plt.plot(r.history['val_accuracy'], label='val_acc')
#plt.legend()

#plt.savefig('Conflict_Class_Accuracy_Predict_20sec.png', dpi=220, bbox_inches='tight')
#---------------------------------------------------

In [ ]:
### Further Train the Network with a smaller learning rate:

model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0003),      ## ?????
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=50,
  validation_split=0.1,        ## ?????
  #validation_split=0.5,
  )


In [ ]:
### Further Train the Network with a smaller learning rate:

model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0003),      ## ?????
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=50,
  validation_split=0.1,        ## ?????
  #validation_split=0.5,
  )


In [ ]:
### Further Train the Network with a smaller learning rate:

model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0003),      ## ?????
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=20,
  validation_split=0.1,        ## ?????
  #validation_split=0.5,
  )


In [ ]:
### Further Train the Network with a smaller learning rate:

model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0003),      ## ?????
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=10,
  validation_split=0.1,        ## ?????
  #validation_split=0.5,
  )


In [ ]:
### Further Train the Network with a smaller learning rate:

model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0002),      ## ?????
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=10,
  validation_split=0.1,        ## ?????
  #validation_split=0.5,
  )


In [ ]:
### Further Train the Network with a smaller learning rate:

model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0001),      ## ?????
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=10,
  validation_split=0.1,        ## ?????
  #validation_split=0.5,
  )


In [ ]:
### Further Train the Network with a smaller learning rate:

model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.00009),      ## ?????
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=10,
  validation_split=0.1,        ## ?????
  #validation_split=0.5,
  )


In [ ]:
### Further Train the Network with a smaller learning rate:

model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.00007),      ## ?????
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.1,        ## ?????
  #validation_split=0.5,
  )


In [ ]:
### Further Train the Network with a smaller learning rate:

model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.00006),      ## ?????
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.1,        ## ?????
  #validation_split=0.5,
  )


In [ ]:
### Further Train the Network with a smaller learning rate:

model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.00005),      ## ?????
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.1,        ## ?????
  #validation_split=0.5,
  )


In [ ]:
### Further Train the Network with a smaller learning rate:

model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.00002),      ## ?????
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.1,        ## ?????
  #validation_split=0.5,
  )


In [ ]:
### Further Train the Network with a smaller learning rate:

model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.000007),      ## ?????
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.1,        ## ?????
  #validation_split=0.5,
  )


In [ ]:
### Further Train the Network with a smaller learning rate:

model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.000001),      ## ?????
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=5,
  validation_split=0.1,        ## ?????
  #validation_split=0.5,
  )


In [ ]:
model.save('/content/drive/My Drive/Pretrained_Model')

In [ ]:
model = tf.keras.models.load_model('/content/drive/My Drive/Pretrained_Model')

In [ ]:
### Further Train the Network with a smaller learning rate:

model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   #optimizer=Adam(lr=0.0001),      ## ?????
   optimizer=SGD(lr=0.0001, momentum=0.1),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r1 = model.fit(
  inputs, Y_train,
  epochs=20,
  validation_split=0.1,        ## ?????
  #validation_split=0.5,
  )


In [ ]:
model.save('/content/drive/My Drive/Pretrained_Model')

In [ ]:
model.summary()

In [ ]:
model.save('/content/drive/My Drive/Pretrained_Model')

In [ ]:
model = tf.keras.models.load_model('/content/drive/My Drive/Pretrained_Model')

In [ ]:
#layer_6.weights

In [ ]:
#layer_6.weights

In [ ]:
### Prepare a data set including misclassified/false detection examples to retrain the RNN 

Y_train_Class = Y_train.flatten()

outputs_train_1 = model.predict(X_train)
outputs_train = outputs_train_1.flatten() 

outputs_train_Class = (outputs_train>0.5)*1

mis_classed = outputs_train_Class != Y_train_Class    #  == or !=   ????

XX_train = np.zeros(shape=(np.sum(mis_classed), T, D))
YY_train = np.zeros(np.sum(mis_classed))

i_xx = 0
for i_m in range(len(mis_classed)):

  if (mis_classed[i_m] == 1) and (Y_train_Class[i_m] == 1):

    XX_train[i_xx] = X_train[i_m]
    YY_train[i_xx] = Y_train_Class[i_m]

    i_xx += 1
  


In [ ]:
YY_train.shape

In [ ]:
XX_train.shape

In [ ]:
### Freeze all the layers but the last

layer_1.trainable = False
layer_2.trainable = False
layer_3.trainable = False
layer_4.trainable = False
layer_5.trainable = False

In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.00002),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r2 = model.fit(
  XX_train, YY_train,
  epochs=10,
  validation_split=0.3,
  #validation_split=0.5,
  )


In [ ]:
# Retrain the whole network with a small training rate

layer_1.trainable = True
layer_2.trainable = True
layer_3.trainable = True
layer_4.trainable = True

In [ ]:
model.compile(
  loss='binary_crossentropy',
  # optimizer='rmsprop',
#   optimizer='adam',
   optimizer=Adam(lr=0.0000001),
   #optimizer=SGD(lr=0.1, momentum=0.9),
  #metrics=['accuracy'],
  metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)],
)

#------------------------------------

# train the RNN
r3 = model.fit(
  XX_train, YY_train,         # Set the data set right
  epochs=10,
  validation_split=0.1,
  #validation_split=0.5,
  )


In [ ]:
Y_train.shape

In [ ]:
YY_train.shape

In [ ]:
#new_model.summary()

In [ ]:
#Data_Frame_10 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_NoisyData_20sec_NEW_10.csv')#, header=None)  

Data_Frame_10 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_20sec_NEW_10.csv')#, header=None)  



In [ ]:
Data_Frame_10.head()

#list(Data_Frame_10)[14]

#Data_Frame_10[list(Data_Frame_10)[14]]


In [ ]:
### Data_Franme_10:

### DETERMINE the number of conflict versus non-conflict case.

#------------------------

#T = 15
#T1 = 15
#D = 12


Y_TEMP_10 = []


col_list_10 = list(Data_Frame_10)
N_enc_train_10 = (np.array(len(col_list_10)-2)/13).astype(int) #- 5


for i_enc in range(N_enc_train_10):    # Number of examples

    if Data_Frame_10[col_list_10[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

      for i_t in range(150):

        y = Data_Frame_10[col_list_10[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

        if (y==0 or y==1) and i_t>T1-2:

            Y_TEMP_10.append(y)


Y_TEMP_10 = np.array(Y_TEMP_10)
len(Y_TEMP_10)


N_ratio_10 = np.sum(Y_TEMP_10)/(len(Y_TEMP_10)-np.sum(Y_TEMP_10))
N_ratio_10

In [ ]:
N_enc_train_10

In [ ]:
# TESTING based on some examples:

X_t = []
Y_t = []

col_list_10 = list(Data_Frame_10)

for i_enc in range(N_enc_train_10):   #[N_enc_train_10+10]:    # Number of examples

    if Data_Frame_10[col_list_10[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

      for i_t in range(150):
        
        #=============================== Prepare for Future classification case

        y = Data_Frame_10[col_list_10[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

        x = np.zeros(shape=(T,D))

        if (y==0 or y==1) and i_t>T1-2:

           if y==1:

              x[:,0] =  Data_Frame_10[col_list_10[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,1] =  Data_Frame_10[col_list_10[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,2] =  Data_Frame_10[col_list_10[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,3] =  Data_Frame_10[col_list_10[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,4] =  Data_Frame_10[col_list_10[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,5] =  Data_Frame_10[col_list_10[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,6] =  Data_Frame_10[col_list_10[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,7] =  Data_Frame_10[col_list_10[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,8] =  Data_Frame_10[col_list_10[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,9] =  Data_Frame_10[col_list_10[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,10] =  Data_Frame_10[col_list_10[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,11] =  Data_Frame_10[col_list_10[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


              X_t.append(x)
              Y_t.append(y)


           elif y==0  and (np.random.uniform(low=0.0,high=1.0)<N_ratio_10*Ratio_Times):  # and i_t<T2:    # this "and i_t<T2" is to have only examples far away **********


              x[:,0] =  Data_Frame_10[col_list_10[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,1] =  Data_Frame_10[col_list_10[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,2] =  Data_Frame_10[col_list_10[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,3] =  Data_Frame_10[col_list_10[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,4] =  Data_Frame_10[col_list_10[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,5] =  Data_Frame_10[col_list_10[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,6] =  Data_Frame_10[col_list_10[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,7] =  Data_Frame_10[col_list_10[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,8] =  Data_Frame_10[col_list_10[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,9] =  Data_Frame_10[col_list_10[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,10] =  Data_Frame_10[col_list_10[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,11] =  Data_Frame_10[col_list_10[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


              X_t.append(x)
              Y_t.append(y)


        #=================================================================
        #-------------------------Delete Variables
           del x
           del y
  
        #-------------------------

    

  
X_test = np.array(X_t)
Y_test = np.array(Y_t)



In [ ]:
N_enc_train_10

In [ ]:
### Delete those variables that are not needed any more to use less RAM

del Data_Frame_10

del Y_TEMP_10

del col_list_10

del N_enc_train_10

del N_ratio_10

%del Y_a_10



In [ ]:
# One-step forecast using true targets

outputs = model.predict(X_test)

#outputs = new_model.predict(X_test)


In [ ]:
outputs.shape

In [ ]:
Y_test

In [ ]:
RNN_Class_1 = outputs.flatten()
RNN_Class_1

In [ ]:
RNN_Class = (RNN_Class_1>0.5)*1
RNN_Class

In [ ]:
Y_test.shape

In [ ]:
RNN_Class.shape

In [ ]:
np.sum(Y_test)

In [ ]:
len(Y_test)-np.sum(Y_test)

In [ ]:
X_test.shape

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, RNN_Class)
print(confusion_matrix)

In [ ]:
P_1g1 = confusion_matrix[1][1]/np.sum(Y_test)
P_0g1 = confusion_matrix[1][0]/np.sum(Y_test)
P_0g0 = confusion_matrix[0][0]/(len(Y_test) - np.sum(Y_test))
P_1g0 = confusion_matrix[0][1]/(len(Y_test) - np.sum(Y_test))

In [ ]:
P_1g1

In [ ]:
P_0g1

In [ ]:
P_0g0

In [ ]:
P_1g0

In [ ]:
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
yhat_classes = model.predict_classes(X_test, verbose=0)

In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Y_test, RNN_Class)
accuracy
#print('Accuracy: %f' % accuracy)

In [ ]:
# precision tp / (tp + fp)
precision = precision_score(Y_test, RNN_Class)
precision
#print('Precision: %f' % precision)


In [ ]:
# recall: tp / (tp + fn)
recall = recall_score(Y_test, RNN_Class)
recall
#print('Recall: %f' % recall)

In [ ]:
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Y_test, RNN_Class)
f1
#print('F1 score: %f' % f1)


In [ ]:
# kappa
kappa = cohen_kappa_score(Y_test, RNN_Class)
kappa
#print('Cohens kappa: %f' % kappa)

In [ ]:
# ROC AUC
auc = roc_auc_score(Y_test, RNN_Class)
auc
#print('ROC AUC: %f' % auc)

In [ ]:
from sklearn import datasets, metrics, model_selection, svm

# ROC AUC
fpr, tpr, thresholds = metrics.plot_roc_curve(Y_test, RNN_Class, pos_label=None)

#print('ROC AUC: %f' % auc)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, RNN_Class))

In [ ]:
N_enc_train_10

In [ ]:
#Data_Frame_11 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_NoisyData_20sec_NEW_10_TestPlot.csv')#, header=None)  

Data_Frame_11 = pd.read_csv('/content/drive/My Drive/MITLL_Correlated_RNN_20sec_NEW_10_TestPlot.csv')#, header=None)  

In [ ]:
Data_Frame_11.head()

In [ ]:
# TESTING based on some examples:

X1_t = []
Y1_t = []

col_list_11 = list(Data_Frame_11)

for i_enc in [3]:    # Number of examples: max is N_enc_train_10

    if Data_Frame_11[col_list_11[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[0]==-1:

      for i_t in range(150):
        
        #=============================== Prepare for Future classification case

        y = Data_Frame_11[col_list_11[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[i_t]

        x = np.zeros(shape=(T,D))

        if (y==0 or y==1) and i_t>T1-2:

           if y==1:

              x[:,0] =  Data_Frame_11[col_list_11[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,1] =  Data_Frame_11[col_list_11[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,2] =  Data_Frame_11[col_list_11[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,3] =  Data_Frame_11[col_list_11[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,4] =  Data_Frame_11[col_list_11[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,5] =  Data_Frame_11[col_list_11[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,6] =  Data_Frame_11[col_list_11[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,7] =  Data_Frame_11[col_list_11[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,8] =  Data_Frame_11[col_list_11[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,9] =  Data_Frame_11[col_list_11[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,10] =  Data_Frame_11[col_list_11[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,11] =  Data_Frame_11[col_list_11[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


              X1_t.append(x)
              Y1_t.append(y)


           elif y==0:


              x[:,0] =  Data_Frame_11[col_list_11[i_enc*13 + 2]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,1] =  Data_Frame_11[col_list_11[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,2] =  Data_Frame_11[col_list_11[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,3] =  Data_Frame_11[col_list_11[i_enc*13 + 5]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,4] =  Data_Frame_11[col_list_11[i_enc*13 + 6]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,5] =  Data_Frame_11[col_list_11[i_enc*13 + 7]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,6] =  Data_Frame_11[col_list_11[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,7] =  Data_Frame_11[col_list_11[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,8] =  Data_Frame_11[col_list_11[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,9] =  Data_Frame_11[col_list_11[i_enc*13 + 11]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,10] =  Data_Frame_11[col_list_11[i_enc*13 + 12]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]
              x[:,11] =  Data_Frame_11[col_list_11[i_enc*13 + 13]].to_numpy(dtype=float, copy=True)[i_t-T+1:i_t+1]


              X1_t.append(x)
              Y1_t.append(y)


        #=================================================================
        #-------------------------Delete Variables
           del x
           del y
  
        #-------------------------


  
X1_test = np.array(X1_t)
Y1_test = np.array(Y1_t)



In [ ]:
### Prepare the above example to plot


x_p = np.zeros(shape=(150,D))

x_p[:,0] = Data_Frame_11[col_list_11[i_enc*13 + 2]].to_numpy(dtype=float, copy=True) 
x_p[:,1] = Data_Frame_11[col_list_11[i_enc*13 + 3]].to_numpy(dtype=float, copy=True)
x_p[:,2] = Data_Frame_11[col_list_11[i_enc*13 + 4]].to_numpy(dtype=float, copy=True)

x_p[:,6] = Data_Frame_11[col_list_11[i_enc*13 + 8]].to_numpy(dtype=float, copy=True)
x_p[:,7] = Data_Frame_11[col_list_11[i_enc*13 + 9]].to_numpy(dtype=float, copy=True)
x_p[:,8] = Data_Frame_11[col_list_11[i_enc*13 + 10]].to_numpy(dtype=float, copy=True)


In [ ]:
### Plot some encounters: Horizontal

#=================
V_P = plt.subplot(111)
#=================    

V_P.plot(x_p[:,0], x_p[:,1], 'b', label='Track_1')    
V_P.plot(x_p[:,6], x_p[:,7], 'r', label='Track_2')    

V_P.plot(x_p[0,0], x_p[0,1], 'Dk')    
V_P.plot(x_p[0,6], x_p[0,7], 'Dk')    

plt.title('Trajectories (Horizontal)', fontsize=15)

#plt.axis([0, 500, 0.0001, 0.05])
pylab.legend(loc='center left')
#pylab.ylim(-1.5, 2.0)

#plt.xlabel('Time (sec)')
plt.xlabel('East (nmi)')
plt.ylabel('North (nmi)')

plt.savefig('Traj_H_exm3.png', dpi=220, bbox_inches='tight')


In [ ]:
### Plot some encounters:  Vertical

#=================
V_P = plt.subplot(111)
#=================    

V_P.plot(x_p[:,2]*1852*3.28084, 'b', label='Track_1')    
V_P.plot(x_p[:,8]*1852*3.28084, 'r', label='Track_2')    

V_P.plot(x_p[0,2]*1852*3.28084, 'Dk')    
V_P.plot(x_p[0,8]*1852*3.28084, 'Dk')    

plt.title('Trajectories (Vertical)', fontsize=15)

#plt.axis([0, 500, 0.0001, 0.05])
pylab.legend(loc='center right')
#pylab.ylim(-1.5, 2.0)

#plt.xlabel('Time (sec)')
plt.xlabel('Time')
plt.ylabel('Up (ft)')

plt.savefig('Traj_V_exm3.png', dpi=220, bbox_inches='tight')


In [ ]:
### Plot some encounters: Horizontal

#=================
V_P = plt.subplot(111)
#=================    


V_P.plot(np.sqrt((x_p[:,0]-x_p[:,6])**2 + (x_p[:,1]-x_p[:,7])**2), 'b', label='Horizontal Distance')    
V_P.plot(np.ones(len(x_p[:,0])), 'r', label='Threshold')    


plt.title('Distance (Horizontal)', fontsize=15)

#plt.axis([0, 500, 0.0001, 0.05])
pylab.legend(loc='center left')
#pylab.ylim(-1.5, 2.0)

#plt.xlabel('Time (sec)')
#plt.xlabel('East (nmi)')
#plt.ylabel('North (nmi)')

#plt.savefig('Traj_H_exm3.png', dpi=220, bbox_inches='tight')


In [ ]:
### Plot some encounters: Vertical

#=================
V_P = plt.subplot(111)
#=================    


V_P.plot(np.abs(x_p[:,2]-x_p[:,8])*1852*3.28084, 'b', label='Vertical Distance')
V_P.plot(np.ones(len(x_p[:,0]))*400, 'r', label='Threshold')    


plt.title('Distance (Vertical)', fontsize=15)

#plt.axis([0, 500, 0.0001, 0.05])
pylab.legend(loc='center left')
#pylab.ylim(-1.5, 2.0)

#plt.xlabel('Time (sec)')
#plt.xlabel('East (nmi)')
#plt.ylabel('North (nmi)')

#plt.savefig('Traj_H_exm3.png', dpi=220, bbox_inches='tight')


In [ ]:
### Plot some encounters: Horizontal

#=================
V_P = plt.subplot(111)
#=================    


V_P.plot((np.sqrt((x_p[:,0]-x_p[:,6])**2 + (x_p[:,1]-x_p[:,7])**2)<1)*1, '.b', label='Horizontal Distance')    
V_P.plot(((np.abs(x_p[:,2]-x_p[:,8])*1852*3.28084)<400)*1, 'ok', label='Vertical Distance')

yy = np.zeros(len(x_p[:,0]))
yy[39:] = Data_Frame_11[col_list_11[i_enc*13 + 14]].to_numpy(dtype=float, copy=True)[19:130]
V_P.plot(yy, 'r', label='Truth')


plt.title('Class', fontsize=15)

#plt.axis([0, 500, 0.0001, 0.05])
pylab.legend(loc='center left')
#pylab.ylim(-1.5, 2.0)

#plt.xlabel('Time (sec)')
#plt.xlabel('East (nmi)')
#plt.ylabel('North (nmi)')

#plt.savefig('Traj_H_exm3.png', dpi=220, bbox_inches='tight')


In [ ]:
#X2_test = X1_test.copy()
#X2_test[0] = X2_test[0] + 10

In [ ]:
# One-step forecast using true targets

outputs_1 = model.predict(X1_test)

#outputs_1 = new_model.predict(X1_test)


In [ ]:
Y1_test.shape

In [ ]:
Y1_test

In [ ]:
RNN_Class_1_1 = outputs_1.flatten()
RNN_Class_1_1

In [ ]:
RNN_Class_th = (RNN_Class_1_1>0.5)*1
RNN_Class_th

In [ ]:
#=================
V_P = plt.subplot(111)
#=================    
V_P.plot(Y1_test, 'Db', label='Truth')    
V_P.plot(RNN_Class_1_1, '.r', label='RNN Class Probability')    
V_P.plot(RNN_Class_th, 'k', label='RNN Class')    


plt.title('Classes and RNN Probability Outputs', fontsize=15)

#plt.axis([0, 500, 0.0001, 0.05])
pylab.legend(loc='center left')
#pylab.ylim(-1.5, 2.0)

#plt.xlabel('Time (sec)')
#plt.ylabel('Class')

plt.savefig('Conflict_Class_Predict_exm3_20sec.png', dpi=220, bbox_inches='tight')


Probabilities obtained based on confusion matrix:

P(1|1) = 0.97627           (first try it was 0.97)

P(0|1) = 0.02372        (first try it was 0.03)

P(0|0) = 0.89637           (first try it was 0.8)

P(1|0) = 0.10362           (first try it was 0.2)


In [ ]:
### Prepare the above example to plot

i_enc_exm = N_enc_train_1+4

x_p = np.zeros(shape=(150,D))

x_p[:,0] = Data_Frame_1[col_list_1[i_enc_exm*13 + 2]].to_numpy(dtype=float, copy=True)
x_p[:,1] = Data_Frame_1[col_list_1[i_enc_exm*13 + 3]].to_numpy(dtype=float, copy=True)
x_p[:,2] = Data_Frame_1[col_list_1[i_enc_exm*13 + 4]].to_numpy(dtype=float, copy=True)

x_p[:,6] = Data_Frame_1[col_list_1[i_enc_exm*13 + 8]].to_numpy(dtype=float, copy=True)
x_p[:,7] = Data_Frame_1[col_list_1[i_enc_exm*13 + 9]].to_numpy(dtype=float, copy=True)
x_p[:,8] = Data_Frame_1[col_list_1[i_enc_exm*13 + 10]].to_numpy(dtype=float, copy=True)


In [ ]:
### Plot some encounters: Horizontal

#=================
V_P = plt.subplot(111)
#=================    

V_P.plot(x_p[:,0], x_p[:,1], '.b', label='Track_1')    
V_P.plot(x_p[:,6], x_p[:,7], '.r', label='Track_2')    

V_P.plot(x_p[0,0], x_p[0,1], 'Dr')    
V_P.plot(x_p[0,6], x_p[0,7], 'Db')    

plt.title('Trajectories (Horizontal)', fontsize=15)

#plt.axis([0, 500, 0.0001, 0.05])
pylab.legend(loc='center right')
#pylab.ylim(-1.5, 2.0)

#plt.xlabel('Time (sec)')
plt.xlabel('East (nmi)')
plt.ylabel('North (nmi)')

#plt.savefig('Traj_H_exm1.png', dpi=220, bbox_inches='tight')


In [ ]:
### Plot some encounters:  Vertical

#=================
V_P = plt.subplot(111)
#=================    

V_P.plot(x_p[:,2]*1852*3.28084, '.b', label='Track_1')    
V_P.plot(x_p[:,8]*1852*3.28084, '.r', label='Track_2')    

V_P.plot(x_p[0,2]*1852*3.28084, 'Dr')    
V_P.plot(x_p[0,8]*1852*3.28084, 'Db')    

plt.title('Trajectories (Vertical)', fontsize=15)

#plt.axis([0, 500, 0.0001, 0.05])
pylab.legend(loc='center right')
#pylab.ylim(-1.5, 2.0)

#plt.xlabel('Time (sec)')
plt.xlabel('Time')
plt.ylabel('Up (ft)')

#plt.savefig('Traj_V_exm1.png', dpi=220, bbox_inches='tight')
